# Seleção de atributos com testes de hipoteses - SelectFdr

Testes estatísticos univariados são aqueles que envolvem uma variável dependente, por exemplo, teste t ou teste z para comparação de médias. 

Proporção esperada de erros do tipo 1. Um erro do tipo 1 é quando a hipotese nula é rejeitada incorretamente, ou seja, é obtido um falso positivo.

- Erro do tipo 1: Quando rejeitamos a hipotese nula;
- Erro do tipo 2: Não rejeitar nula quando deveria ter rejeitado.

In [1]:
import pandas as pd 
import numpy as np
from sklearn.feature_selection import SelectFdr
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import chi2 

In [2]:
dataset = pd.read_csv("ad.data", header=None)
dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,1549,1550,1551,1552,1553,1554,1555,1556,1557,1558
0,125,125,1.0000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
1,57,468,8.2105,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
2,33,230,6.9696,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
3,60,468,7.8000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
4,60,468,7.8000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.


In [3]:
dataset.shape

(3279, 1559)

Essa base de dados é definida para identificar o que é um post de anuncio ou não na internet

In [4]:
X = dataset.iloc[:,0:1558].values
X

array([[125.    , 125.    ,   1.    , ...,   0.    ,   0.    ,   0.    ],
       [ 57.    , 468.    ,   8.2105, ...,   0.    ,   0.    ,   0.    ],
       [ 33.    , 230.    ,   6.9696, ...,   0.    ,   0.    ,   0.    ],
       ...,
       [ 23.    , 120.    ,   5.2173, ...,   0.    ,   0.    ,   0.    ],
       [  0.    ,   0.    ,   0.    , ...,   0.    ,   0.    ,   0.    ],
       [ 40.    ,  40.    ,   1.    , ...,   0.    ,   0.    ,   0.    ]])

In [5]:
y = dataset.iloc[:, 1558].values
y

array(['ad.', 'ad.', 'ad.', ..., 'nonad.', 'nonad.', 'nonad.'],
      dtype=object)

In [6]:
np.unique(y, return_counts=True)

(array(['ad.', 'nonad.'], dtype=object), array([ 459, 2820], dtype=int64))

Logo, percebemos que existem mais determinações quando não é anuncio. 
Para balancear poderíamos aplicar algumas técnicas para essa base de dados desbalanceada. 

In [7]:
naive1 = GaussianNB()
naive1.fit(X, y)
previsoes1 = naive1.predict(X)
accuracy_score(y, previsoes1)

0.7813357731015553

In [11]:
selecao = SelectFdr(chi2, alpha=0.01) # alpha indica a confiabilidade, probabilidade de rejeitar a hipotese nula
X_novo = selecao.fit_transform(X, y) # Selecionando as melhores caracteristicas que definem o modelo

In [12]:
X.shape, X_novo.shape

((3279, 1558), (3279, 433))

In [14]:
selecao.pvalues_, len(selecao.pvalues_)

(array([2.14710304e-268, 0.00000000e+000, 8.98165813e-150, ...,
        6.03353380e-041, 5.63437216e-012, 9.37945775e-002]),
 1558)

In [15]:
np.sum(selecao.pvalues_ <= 0.01)

476

In [16]:
colunas = selecao.get_support()
colunas

array([ True,  True,  True, ...,  True,  True, False])

As colunas selecionadas é true, e as não selecionadas é false.

In [19]:
indices = np.where(colunas == True)
indices

(array([   0,    1,    2,    9,   11,   14,   20,   21,   26,   31,   34,
          36,   49,   58,   59,   64,   65,   69,   70,   86,   91,   95,
          96,  102,  104,  106,  113,  133,  134,  139,  155,  158,  163,
         167,  172,  175,  180,  181,  183,  185,  186,  189,  190,  192,
         193,  224,  242,  246,  248,  249,  251,  259,  264,  265,  266,
         267,  268,  269,  270,  274,  276,  278,  287,  290,  293,  304,
         307,  310,  312,  317,  321,  329,  330,  336,  341,  345,  346,
         350,  351,  355,  356,  357,  359,  366,  367,  370,  372,  381,
         386,  388,  389,  398,  405,  418,  420,  426,  427,  429,  430,
         432,  435,  440,  455,  457,  460,  465,  470,  472,  477,  478,
         482,  508,  511,  518,  528,  532,  533,  540,  542,  548,  551,
         556,  572,  573,  574,  576,  586,  622,  625,  627,  638,  643,
         648,  653,  658,  661,  663,  666,  683,  688,  694,  703,  704,
         709,  711,  720,  723,  729, 

In [20]:
naive2 = GaussianNB()
naive2.fit(X_novo, y)
previsoes2 = naive2.predict(X_novo)
accuracy_score(y, previsoes2)

0.970722781335773

Dessa forma, selecionamos os melhores atributos que definem o modelo a partir da técnica de feature selection, baseado no teste Chi2. 

Por fim o modelo deve um ganho de performance absurdo, mas é importante ressaltar que seria necessário realizar a divisão da bases de dados entre treino e teste, validação com as demais métricas. Aqui foi somente para elucidar o chi2. 